In [14]:
from sklearn.model_selection import train_test_split

import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import tensorflow as tf
import IPython.display as ipd
import random
from pydub import AudioSegment
import import_ipynb
from models.cnn import CNN_model
from service.Mfcc import get_mfcc

In [7]:
filename = 'labels_spotter'
infile = open(filename,'rb')
lb = pickle.load(infile)
infile.close()

In [8]:
model_cnn = tf.keras.models.load_model('saved_models/CNN')
model_crnn = tf.keras.models.load_model('saved_models/CRNN')
model_at_rnn = tf.keras.models.load_model('saved_models/AT_RNN')
model_ds_cnn = tf.keras.models.load_model('saved_models/DS_CNN')
#model_dnn = tf.keras.models.load_model("saved_models/DNN")

In [36]:
def mfcc_predict(flags, mfcc):
    mfcc = (mfcc - flags['mean']) / flags['std']
    mfcc = np.array(mfcc)
    if flags['model'] == 'CNN':
        prediction = model_cnn.predict(mfcc, batch_size=16, verbose=1)
    elif flags['model'] == 'CRNN':
        prediction = model_crnn.predict(mfcc, batch_size=16, verbose=1)
    elif flags['model'] == 'AT_RNN':
        prediction = model_at_rnn.predict(mfcc, batch_size=16, verbose=1)
    elif flags['model'] == 'DNN':
        prediction = model_dnn.predict(mfcc, batch_size=16, verbose=1)
    elif flags['model'] == 'DS_CNN':
        prediction = model_dc_cnn.predict(mfcc, batch_size=16, verbose=1)
    max_p = prediction.max(axis=1)
    max_index = prediction.argmax(axis=1)
    return max_p, lb.classes_[max_index][0]

In [15]:
filename = 'Data_spotter_mfcc_40'
outfile = open(filename,'rb')
X = pickle.load(outfile)
outfile.close()

In [16]:
df = pd.read_csv('../../full_df')

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X
                                                    , df.word
                                                    , test_size=0.1
                                                    , shuffle=True
                                                    , random_state=42)

In [18]:
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

In [19]:
flags_1 = {
    'mean': mean,
    'std': std,
    'model': 'AT_RNN'
}

In [20]:
flags_2 = {
    'offset': 0,
    'duration': 1,
    'sample_rate': 44100,
    'n_mfcc': 40,
    'columns': 61
}

In [54]:
mfcc = get_mfcc('../../2/yes.oga', flags_2)

In [55]:
res = mfcc_predict(flags_1, mfcc)

1/1 [==============================] - 0s 42ms/step


In [56]:
res

(array([0.9998791], dtype=float32), 'yes')

In [1]:
pwd

'C:\\Users\\Mi Notebook\\Desktop\\Audio_project\\audio_project\\Spotter_recognition'

In [27]:
lb.classes_

array(['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop',
       'unknown', 'up', 'yes'], dtype=object)